## Testing notebook
<br/>
it aims to test each files and function 
<br/>  
<br/>

Overview :   
- MRI_loader.py 
- registration_loader.py
- utils.py 
- losses.py
- Model.py


<br/>   
  
### MRI_loader

In [1]:
from config import dataloader_config

In [2]:
from Dataloaders.MRI_loader import get_labels_list
print(get_labels_list(dataloader_config['rootdir']))

['003374.jpeg', '009084.jpeg', '002772.jpeg', '006854.jpeg', '000902.jpeg', '007515.jpeg', '000228.jpeg', '003343.jpeg', '002384.jpeg', '004192.jpeg', '005287.jpeg', '005847.jpeg', '005680.jpeg', '003612.jpeg', '003272.jpeg', '001171.jpeg', '005352.jpeg', '007357.jpeg', '005415.jpeg', '002355.jpeg', '004263.jpeg', '000535.jpeg', '008958.jpeg', '005892.jpeg', '006660.jpeg', '007307.jpeg', '007047.jpeg', '004996.jpeg', '007671.jpeg', '005743.jpeg', '004391.jpeg', '003229.jpeg', '000053.jpeg', '003599.jpeg', '009080.jpeg', '003023.jpeg', '001599.jpeg', '004867.jpeg', '000397.jpeg', '001734.jpeg', '000364.jpeg', '006284.jpeg', '001826.jpeg', '007447.jpeg', '005478.jpeg', '003456.jpeg', '003237.jpeg', '003977.jpeg', '009828.jpeg', '001974.jpeg', '001235.jpeg', '001435.jpeg', '000622.jpeg', '006845.jpeg', '006450.jpeg', '003568.jpeg', '002017.jpeg', '000912.jpeg', '003075.jpeg', '007867.jpeg', '000644.jpeg', '004538.jpeg', '008144.jpeg', '009407.jpeg', '005671.jpeg', '003106.jpeg', '008444.j

In [3]:
from Dataloaders.MRI_loader import get_data_dicts

partition=get_data_dicts(dataloader_config['rootdir'],dataloader_config['valid_ratio'])
print(len(partition['train']),'\n',len(partition['validation']),'\n',partition['train'][8])

8000 
 2001 
 /home/luther/Documents/Projet_3A/data/2D_data/MedNIST/Hand/002384.jpeg


In [4]:
from Dataloaders.MRI_loader import load

fixed_image,moving_image=load(partition['train'][0],partition['train'][1],vectorize=False)
fixed_image.shape

(64, 64)

We also can see above if to_vector from process.utils works
### registration_loader

In [5]:
from Dataloaders.registration_loader import Registration_dataset
from torchvision import transforms
from monai.transforms import AddChannel


t = transforms.Compose([transforms.ToTensor(), AddChannel()])
            
dataset=Registration_dataset(partition['train'], t, vectorize=False)
fixed_image, moving_image=dataset[2]
fixed_image.shape

/home/luther/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:114: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


torch.Size([1, 1, 64, 64])

In [6]:
from Dataloaders.registration_loader import get_dataloaders
training_generator, validation_generator=get_dataloaders(dataloader_config)
next(iter(training_generator))[0].shape

torch.Size([1, 1, 64, 64])

La fonction addchannel de monai permet d'ajouter la dimension channel sans faire un .view dans le training
<br/>
### Utils
<br/>

Vectorization

In [7]:
from process.utils import to_vector
#Currently 2D data but with 3D how would it works ?
fixed_image,moving_image,fixed_mask,moving_mask=load(partition['train'][0],partition['train'][1])
to_vector(fixed_image).shape

(160, 192, 224, 3)

### Losses 
<br/>

#### compute dice test

In [8]:
from process.losses import compute_dice

#let's compare 2 mask from two random image of our dataset
fixed_image,moving_image,fixed_mask,moving_mask=next(iter(training_generator))
compute_dice(fixed_mask, moving_mask)

tensor([[0.4990, 0.6892, 0.6315, 0.3801]])

MSE

In [7]:
from process.losses import MSE

#let's compare 2 images from dataset
MSE(fixed_image, moving_image)

tensor(0.1312)

Gradient

In [12]:
from process.losses import Grad
import torch

flow_field_random=torch.abs(torch.randn([1,2,64,64])) 
Grad('l2').loss(flow_field_random)

tensor(0.4947)

### Model

<br/>


In [10]:
#model initialization
from Model import Model
from config import model_config, criterion_config, scheduler_config

test_model=Model(model_config, criterion_config,scheduler_config)
test_model.net

VxmDense(
  (unet_model): Unet(
    (encoder): ModuleList(
      (0): ModuleList(
        (0): ConvBlock(
          (main): Conv2d(2, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation): LeakyReLU(negative_slope=0.2)
        )
      )
      (1): ModuleList(
        (0): ConvBlock(
          (main): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation): LeakyReLU(negative_slope=0.2)
        )
      )
      (2): ModuleList(
        (0): ConvBlock(
          (main): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation): LeakyReLU(negative_slope=0.2)
        )
      )
      (3): ModuleList(
        (0): ConvBlock(
          (main): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation): LeakyReLU(negative_slope=0.2)
        )
      )
    )
    (decoder): ModuleList(
      (0): ModuleList(
        (0): ConvBlock(
          (main): Conv2d(32, 32, kernel_size=(3, 3

In [11]:
#model forward
fixed_image,moving_image=fixed_image.to(test_model.device),moving_image.to(test_model.device)
pred_image,flow_field=test_model.net(moving_image.float(), fixed_image.float())
pred_image.shape,flow_field.shape

/home/luther/.local/lib/python3.8/site-packages/torch/nn/functional.py:3657: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(


(torch.Size([1, 1, 64, 64]), torch.Size([1, 2, 32, 32]))

In [14]:
#model interpolation
moving_mask=moving_mask.to(test_model.device)
pred_mask = test_model.net.transformer(moving_mask.float(), flow_field)
pred_mask.shape

NameError: name 'flow_field' is not defined

In [13]:
#model criterion
import torch
flow_field_random=torch.abs(torch.randn([1,3,160,192,224])) 
loss = test_model.criterion(moving_image, fixed_image, flow_field_random)
print(loss)

tensor(0.1676)


### Trainer

In [16]:
from torch import load, save 
from Model import Model
from Trainer import Trainer
from config import model_config, dataloader_config, train_config

#model = Model(model_config)
trainer = Trainer(test_model, dataloader_config, train_config)
trainer.trainloader

In [17]:
trainer.train()

 14%|█████▌                                 | 1129/8000 [00:48<04:57, 23.12it/s]


KeyboardInterrupt: 

In [18]:
trainer.test()

 12%|████▉                                   | 246/2001 [00:03<00:28, 62.36it/s]


KeyboardInterrupt: 

In [19]:
trainer.verbose()


Train Loss................: 0.00
Test Loss.................: 0.00

Current Learning Rate.....: 0.001000


In [5]:
trainer.update_state()

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 19.76it/s]


In [6]:
trainer.verbose()


Train Loss................: 0.19
Test Loss.................: 0.07
Train Mean Dice............: 0.00
Test Mean Dice.............: 0.00

Best Test Mean Dice........: 2.81


#### Model conversion 

In [ ]:
mmconvert -sf tensorflow -in imagenet_resnet_v2_152.ckpt.meta -iw imagenet_resnet_v2_152.ckpt --dstNodeName MMdnn_Output -df pytorch -om tf_resnet_to_pth.pth

In [ ]:
mmconvert -sf keras -iw vxm_dense_brain_T1_3D_mse.h5 -df pytorch -om keras_to_torch.pt